# CS 584 :: Data Mining :: George Mason University :: Fall 2022


# Homework 3: Clustering&Association Rule Mining

- **100 points [9% of your final grade]**
- **Due Friday, November 4 by 11:59pm**

- *Goals of this homework:* (1) implement your K-means model; and (2) implement the association rule mining process with the Apriori algorithm.

- *Submission instructions:* for this homework, you only need to submit to Blackboard. Please name your submission **FirstName_Lastname_hw3.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw3.ipynb**. Your notebook should be fully executed so that we can see all outputs. 

## Part 1: Clustering (50 points)

In this part, you will implement your own K-means algorithm to conduct clustering on handwritten digit images. In this homework, we will still use the handwritten digit image dataset we have already used in previous homework. However, since clustering is unsupervised learning, which means we do not have the label information. So, here, we will only use the testing data stored in the "test.txt" file.

First, let's load the data by excuting the following code:

In [4]:
import numpy as np
import math
import sys

test = np.loadtxt("test.txt", delimiter=',')
test_features = test[:, 1:]
test_labels = test[:, 0]
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing label matrix: shape ' + str(np.shape(test_labels)))

array of testing feature matrix: shape (10000, 784)
array of testing label matrix: shape (10000,)


Now, it's time for you to implement your own K-means algorithm. First, please write your code to build your K-means model using the image data with **K = 10**, and **Euclidean distance**.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn**

**Note: you need to decide when to stop the model training process.**

In [5]:
class kmeansfunctions:
    def __init__(self,data, k):
        self.K = k # number of clusters
        self.noofsamples, self.nooffeatures = data.shape #number of examples, number of features
        
    #function to initialize random centroids

    
    def index_of_minval(self,x):
        return np.argmin(x)                  #this returns the indices of lowest value    
    
    
    def cluster_prediction(self,main_clusters,data):
        cluster_predictions = np.zeros(self.noofsamples) # taking rows and columns as zeros
        for id_cluster, values_of_cluster in enumerate(main_clusters):
            for values in values_of_cluster:
                cluster_predictions[values] = id_cluster
        return cluster_predictions    
    
    # function to create new clusters
    def form_clusters(self,data, centroids):        # this function predicts clusters
        clusters_list = [[] for _ in range(self.K)]
        for pointid, point in enumerate(data):
            closestcentroid = self.index_of_minval(np.sqrt(np.sum((point-centroids)**2, axis=1)))#using eulers distance for calculating centroid values
            clusters_list[closestcentroid].append(pointid)
        return clusters_list
        
    
    def random_centroid_points(self, data):
        centroid_finlist = np.zeros((self.K, self.nooffeatures))  
        for k in range(self.K):
            centroid_value = data[np.random.choice(range(self.noofsamples))] # random centroids
            centroid_finlist[k] = centroid_value
        return centroid_finlist 
    


    # updating the initial centroids
    def updating_centroids(self,main_cluster,data):
        updated_centroids = np.zeros((self.K, self.nooffeatures)) # taking rows and columns as zeros
        for idx, main_cluster in enumerate(main_cluster):
            new_centroid_value = np.mean(data[main_cluster], axis=0) # find the value for new centroids
            updated_centroids[idx] = new_centroid_value
        return updated_centroids
    
                                            
    
        
    # function to fit data
    def fitting_data(self,data): 
        centroids = self.random_centroid_points(data)   #calling function to generate random centroids
        for _ in range(100):
            clusters = self.form_clusters(data, centroids) 
            prevcentroids = centroids
            centroids = self.updating_centroids(clusters,data) 
            change = centroids - prevcentroids   #checking the difference values if there is any change or not
            if not change.any():
                break             #if new centroid -oldcentroids has no change then breaking the loop 
        predictions = self.cluster_prediction(clusters,data) 
        return predictions
            
if __name__ == "__main__":
    k = 10                        #k indicates the number of clusters
    data = test_features
    kmeans_object = kmeansfunctions(data, k)
    final_predictions = kmeans_object.fitting_data(data)
    cent = kmeans_object.random_centroid_points(data)
    cl = kmeans_object.form_clusters(data,cent)
    newcent = kmeans_object.updating_centroids(cl,data)
    print("shape of updated centroids")
    print(newcent.shape)     #printing shape of final centroids
    #print(new_cent[:1,:])
    #print (cl[:1])
    print("total number of clusters are")  #printing the total number of clusters
    print (len(cl))

  

   



shape of updated centroids
(10, 784)
total number of clusters are
10


In [6]:
print(final_predictions)

[6. 8. 1. ... 6. 7. 2.]


Next, you need to calculate the Sum of Squared Error (SSE) of each cluster generated by your K-means algorithm. Then, print out the averaged SSE of your algorithm.

In [7]:
# Sum of Squared Error calculation
finalsse=0      #variable to store final sse
final_clusters=cl
final_centroids=newcent
c=0
for j in range(10):
     for i in final_clusters[j]:
        sse=np.sum((final_centroids[c]-test_features[i])**2)  #sse formula
        finalsse=finalsse+sse
   #print(c)
     c=c+1
print(finalsse/10)


2745061415.729097


Then, please have a look on https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_completeness_v_measure.html#sklearn.metrics.homogeneity_completeness_v_measure, and use this function to print out the homogeneity, completeness, and v-measure of your K-means model.

In [9]:
# Write your code here
import sklearn
from sklearn.metrics import homogeneity_completeness_v_measure
h,c,vm = sklearn.metrics.homogeneity_completeness_v_measure(test_labels, final_predictions)
print ('homogenity:', h)
print ('completeness:', c)
print ('v-measure:', vm)


homogenity: 0.5298832148456388
completeness: 0.5451159984187467
v-measure: 0.537391682043727


Ok, now you already have a good model. But can you further improve it? In the next cell, please implement the K-means++ model introduced in the lecture.

In [11]:
# Write your code here
class kmeans_plus:
    def __init__(self,test, k):
        self.K = k # number of clusters
        self.noofsamples, self.nooffeatures = test.shape # number of examples and number of features
        

    
    # creating clusters
    def form_cluster(self,test, centroids):
        clusters = [[] for _ in range(self.K)]
        for pointid, point in enumerate(test):
            closestcentroid = np.argmin(np.sqrt(np.sum((point-centroids)**2, axis=1)))
            # closest centroid using euler distance equation(calculate distance of every point from centroid)
            clusters[closestcentroid].append(pointid)
        return clusters 
    
    # new centroids
    def new_centroids(self, cluster,test):
        centroids = np.zeros((self.K, self.nooffeatures)) # row , column full with zero
        for idx, cluster in enumerate(cluster):
            newcentroid = np.mean(test[cluster], axis=0) # find the value for new centroids
            centroids[idx] = newcentroid
        return centroids
    
    def initial_random_centroids(self,test):   #this function generates random initial centriods fo kmeans++
        centroids_list = []
        centroids_list.append(test[np.random.randint(test.shape[0]),:])#taking random value points for centroid
        for i in range(k-1):             
            final_dist = []
            for j in range(test.shape[0]):            
                pt = test[j,:]
                d = sys.maxsize
                for z in range(len(centroids_list)):
                    distance = np.sqrt(np.sum((pt-centroids_list[z])**2))  #calculating the distance from centroid to data points
                    d = min(d,distance)
                final_dist.append(d)
            final_dist = np.array(final_dist)
            nextcentroid = test[np.argmax(final_dist),:]
            centroids_list.append(nextcentroid)
            final_dist = []
        return centroids_list    
    
    
    # function to predict
    def predict(self, clusters,test):
        prediction_result = np.zeros(self.noofsamples) 
        for indx, cluster_value in enumerate(clusters):
            for j in  cluster_value:
                prediction_result[j] = indx
        return prediction_result
    
        
    # function to fit data
    def fit_model(self,test):
        # initialize random centroids
        centroids = self.initial_random_centroids(test) 
        for _ in range(100):
            clusters = self.form_cluster(test, centroids) 
            prev = centroids
            updated_centroids = self.new_centroids(clusters,test) 
            change = updated_centroids - prev
            if not change.any():
                break
        predictions = self.predict(clusters,test) 
        return predictions
            
if __name__ == "__main__":
    #np.random.seed(10)
    k = 10 # num of cluster
    test = test_features
    kmean_model = kmeans_plus(test, k)
    final_prediction = kmean_model.fit_model(test)
    centriods = kmean_model.initial_random_centroids(test)
    clusters = kmean_model.form_cluster(test,centriods)
    newcent = kmean_model.new_centroids(clusters,test)
    print(newcent.shape)
    #print(new_cent[:1,:])
    #print (cl[:1])
    print (len(clusters))
    print(final_prediction)
    
    


(10, 784)
10
[7. 0. 4. ... 7. 9. 0.]


In the next cell, use sklearn.metrics.homogeneity_completeness_v_measure() to print out the homogeneity, completeness, and v-measure of your K-means++ model.

In [12]:
import sklearn
from sklearn.metrics import homogeneity_completeness_v_measure
h,c,vm = sklearn.metrics.homogeneity_completeness_v_measure(test_labels, final_prediction)
print ('homogenity:', h)
print ('completeness:', c)
print ('v-measure:', vm)

homogenity: 0.23624960463842856
completeness: 0.2747239469806676
v-measure: 0.2540382908400449


### Question: Comparing the results by K-means and K-means++, do you see significant improvement? Write down in the next cell what do you think are the reasons for observing the improvement or not observing the improve.

#### Write your answer here

## Part 2: Association Rule Mining (50 points)

In this part, you are going to examine movies using our understanding of association rules. For this part, you need to implement the apriori algorithm, and apply it to a movie rating dataset to find association rules of user-rate-movie behaviors. First, run the next cell to load the dataset we are going to use.

In [11]:
user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))

array of user-movie matrix: shape (11743, 2)


In this dataset, there are two columns: the first column is the integer ids of users, and the second column is the integer ids of movies. Each row denotes that the user of given user id rated the movie of the given movie id. We are going to treat each user as a transaction, so you will need to collect all the movies that have been rated by a single user as a transaction. 

Now, you need to implement the apriori algorithm and apply it to this dataset to find association rules of user rating behaviors with **minimum support of 0.2** and **minimum confidence of 0.8**. We know there are many existing implementations of apriori online (check github for some good starting points). You are welcome to read existing codebases and let that inform your approach. 

**Note: Do not copy-paste any existing code.**

**Note: We want your code to have sufficient comments to explain your steps, to show us that you really know what you are doing.**

**Note: You should add print statements to print out the intermediate steps of your method -- e.g., the size of the candidate set at each step of the method, the size of the filtered set, and any other important information you think will highlight the method.**

In [13]:
import numpy as np
import pandas as pd
user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')  #loading and saving data from movie_rated.txt to user_movie_data
daf=pd.read_csv('movies.csv')            #reading movie ids and names from movies.csv
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))
user=user_movie_data[:,:1]     
movies=user_movie_data[:,1:]
updated_movie_list=pd.DataFrame(user_movie_data)   #converting user_movie_data to data frame
modifiedmovies=[]    #list to store movie names and mapping them in order according to movie ids in movie_rated.txt
for i in movies:
    temp=int(i)
    for ind in daf.index:
        temp1=daf['movieId'][ind]
        if(temp1==temp):   #condition to check if movie id matches if it matches replacing movie id with movie name
            add=daf['movie_name'][ind]
            modifiedmovies.append(add)  #adding movie name to list which matches with movie id in movie_rated.txt
#print(modifiedmovies) 
updated_movie_list[1]=modifiedmovies  #updating the usermoviedata frame

#print(user_movie_data)
#print("g")
import itertools
print(updated_movie_list)
final_movie_list=updated_movie_list.values.tolist()  #converting dataframe to list
df={}
for key,val in final_movie_list:
     df.setdefault(key, []).append(val)      #saving values to dictionary

mainlist=[]
j=0
for i in range(1,495):
    mainlist.append(df[i])            #appending values from dictionary to mainlist variable
    j+=1
 

 #print(mainlist)    
confidence = 0.8
support = 0.2                         #taking support and confidence values as 0.2,0.8

dict_list = {}
total_transact = 0
dlist = []
tlist = []
datalist=mainlist
for val_list in datalist:
    tlist = []                    #iterating through datalist which contains users and user rated movie names
    total_transact += 1
    for val in val_list:       #iterating through individual movie names
        tlist.append(val) 
        if val not in dict_list.keys(): 
            dict_list[val] = 1
        else:
            c_value = dict_list[val]
            dict_list[val] = c_value + 1
    dlist.append(tlist)

List_one = []
for key in dict_list:
    if (100 * dict_list[key]/total_transact) >= support*100:  #checking if support is greater than 0.2
        list = []
        list.append(key)
        List_one.append(list)   #appending frequent itemset 1 to List_one
print ("                                            FREQUENT 1-ITEMSET            ")
print (List_one)
print ("  ")




def not_freq_subset(temp,dat_list_one, k):  #function returns true if its not a frequent subset
    item_list = []
    item_list = search_subset(temp,k)   #this is used to iterate through subset
    for item in item_list: 
        sort_list = []              
        for l in item:
            sort_list.append(l)
        sort_list.sort()
        if sort_list not in dat_list_one:    #return true if not a frequent subset
            return True
    return False

def search_subset(ab,cd):
    return set(itertools.combinations(ab,cd))     #searching for subset
 
def main_apriori(dat_list_one, k):
    k_len = k                          
    aprior_list = []    
    for list1 in dat_list_one:  #iterating through dat_list_one
        for list2 in dat_list_one:                 
            c_value = 0
            temp = []
            if list1 != list2:      #checking if list1 is not equal to list2
                while c_value < k_len-1:
                    if list1[c_value] != list2[c_value]:
                        break
                    else:
                        c_value += 1
                else:
                    if list1[k_len-1] < list2[k_len-1]:
                        for item in list1:
                            temp.append(item)
                        temp.append(list2[k_len-1])
                        if not not_freq_subset(temp,dat_list_one, k):    #checking if data is not a frequent sub set
                            aprior_list.append(temp) 
                            temp = []
    return aprior_list



def gen_freq_subsets():
    k = 2
    dat_list_one = []
    fq_item = []          #this function is used to generate frequent item set
    frequent_item_list = []
    c_value = 0
    total_transact = 0
    for item in List_one:
        dat_list_one.append(item)
    while dat_list_one != []:
        aprior_list = []
        fq_item = []
        aprior_list = main_apriori(dat_list_one, k-1)  #calling main apriori function
        
        for i in aprior_list:         #iterating through aprior list
            c_value = 0
            total_transact = 0
            j = set(i)
            for tlist in dlist:
                total_transact += 1
                t_val_item = set(tlist)
                if j.issubset(t_val_item) == True:
                    c_value += 1
            if (100 * c_value/total_transact) >= support*100:  #checking if support is greater than 0.2
                i.sort()
                fq_item.append(i)       #if support is greater append to frequent item list
        dat_list_one = []
        print ("                                         FREQUENT %d-ITEMSET             " % k)
        print (fq_item)             #printing frequent item set list
        print ("                                          ")
        for l in fq_item:
            dat_list_one.append(l)
        k += 1
        if fq_item != []:
            frequent_item_list.append(fq_item)      #if frequent item set is not in list appending the itemset to final frequent item list
    
    return frequent_item_list
     
        


array of user-movie matrix: shape (11743, 2)
           0                                1
0        1.0           Rosemary's Baby (1968)
1        1.0  Children of a Lesser God (1986)
2        1.0    Brothers McMullen, The (1995)
3        1.0             Jurassic Park (1993)
4        2.0           Rosemary's Baby (1968)
...      ...                              ...
11738  494.0   Star Trek: Insurrection (1998)
11739  494.0            Wild Wild West (1999)
11740  494.0          Schindler's List (1993)
11741  494.0           Lethal Weapon 3 (1992)
11742  494.0                      Dune (1984)

[11743 rows x 2 columns]
                                            FREQUENT 1-ITEMSET            
[['Jurassic Park (1993)'], ['Godfather: Part II, The (1974)'], ['Back to the Future (1985)'], ['Groundhog Day (1993)'], ['Princess Bride, The (1987)'], ['Star Wars: Episode IV - A New Hope (1977)'], ['Rain Man (1988)'], ["Schindler's List (1993)"], ['Saving Private Ryan (1998)'], ['Who Framed Roger Ra

Finally, print your final association rules in the following format:

**movie_name_1, movie_name_2, ... --> movie_name_k**

where the movie names can be fetched by joining the movieId with the file 'movies.csv'. For example, one rule that you should find is:

**Jurassic Park (1993), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)**

**Hint: You may need to use the Pandas library to load and process the movies.csv file, such as use pandas.read_csv() to load the data. https://pandas.pydata.org/pandas-docs/dev/user_guide/10min.html is a good place to learn the basics about Pandas.**

In [16]:
# Write your code to print out the rules

def associationrules():    #this function is used to generate associationrules
    item_rel = []
    subsets_list = []
    k_len = 0
    c_value = 1
    increment_one = 0
    increment_val_two = 0
    rule_number = 1
    subset_rel = []
    freq_itemset_val= gen_freq_subsets()   #calling function and storing frequent itemsets
    print ("                              Association rules                    ")
    print(" ")
    for freq_val_record in freq_itemset_val:     #iterating through frequent item record
        for freq_val in freq_val_record:      #iterating through data in frequent item record
            k_len = len(freq_val)
            c_value = 1
            while c_value < k_len: 
                item_rel = []
                subsets_list = search_subset(freq_val,c_value)      
                c_value += 1
                for item in subsets_list:       #iterating through subsets_list
                    increment_one = 0        
                    increment_val_two = 0             
                    item_rel = []
                    subset_rel = []
                    for i in item:    #iterating through item
                        item_rel.append(i)
                    for tlist in dlist:   
                        if set(item_rel).issubset(set(tlist)) == True:
                            increment_one += 1
                        if set(freq_val).issubset(set(tlist)) == True:
                            increment_val_two += 1
                    if 100*increment_val_two/increment_one >= confidence*100: #checking if confidence is greater than 0.8
                        for match_freq in freq_val:
                            if match_freq not in item_rel:
                                subset_rel.append(match_freq)  #appending the itemset with relation to subset_rel
                        print (" %d Rule : %s --> %s" %(rule_number,item_rel,subset_rel))
                        rule_number =rule_number+1  

associationrules()   


                                         FREQUENT 2-ITEMSET             
[['Jurassic Park (1993)', 'Princess Bride, The (1987)'], ['Jurassic Park (1993)', 'Star Wars: Episode IV - A New Hope (1977)'], ['Jurassic Park (1993)', "Schindler's List (1993)"], ['Jurassic Park (1993)', 'Saving Private Ryan (1998)'], ['Jurassic Park (1993)', 'Speed (1994)'], ['Godfather: Part II, The (1974)', 'Star Wars: Episode IV - A New Hope (1977)'], ['Back to the Future (1985)', 'Jurassic Park (1993)'], ['Back to the Future (1985)', 'Groundhog Day (1993)'], ['Back to the Future (1985)', 'Princess Bride, The (1987)'], ['Back to the Future (1985)', 'Star Wars: Episode IV - A New Hope (1977)'], ['Back to the Future (1985)', "Schindler's List (1993)"], ['Back to the Future (1985)', 'Saving Private Ryan (1998)'], ['Back to the Future (1985)', 'Who Framed Roger Rabbit? (1988)'], ['Back to the Future (1985)', 'Godfather, The (1972)'], ['Groundhog Day (1993)', 'Jurassic Park (1993)'], ['Groundhog Day (1993)', 'Pri